In [1]:
import glob
from collections import defaultdict
from pprint import pprint
import pickle
import operator
import os
import nltk
import sklearn
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams
import random
import re
from sklearn.model_selection import GridSearchCV
from IPython.core.display import display, HTML

In [11]:
orig_paper_predicate_expr = set(["great", "so much fun", "good", "so happy", "better", "my favorite thing", "cool", "funny", "nice", "always fun", "fun", "awesome", "the best feeling", "amazing", "happy", "ready today", "ready", "dry", "juicy", "my favorite part"])
orig_paper_pos_expr = set(["love", "missed", "loves", "enjoy", "cant wait", "excited", "wanted", "can't wait", "get", "appreciate", "decided", "loving", "really like", "looooove", "just keeps", "loveee", "randomly stop", "cannot wait", "just live", "please keep", "live", "stoked", "goin", "reading", "break", "just stops", "stops"])
orig_paper_neg_expr = set(["being ignored", "being sick", "waiting", "feeling", "waking up early", "being woken", "fighting", "staying", "writing", "being home", "cleaning", "not getting", "crying", "sitting at home", "being stuck", "starting", "being told", "being left", "getting ignored", "being treated", "doing homework", "learning", "getting up early", "going to bed", "getting sick", "riding", "being ditched", "getting ditched", "missing", "not sleeping", "not talking", "trying", "falling", "walking home", "getting yelled", "being awake", "being talked", "taking care", "doing nothing", "wasting", "throwing", "getting woken up", "to spend", "standing", "smelling", "getting woken", "arguing", "paying bills", "being locked", "shoveling", "getting called", "being at work", "having nothing", "getting invited", "getting blown", "dealing", "ending", "to wake", "when doesn't text", "getting ready", "to learn", "picking", "walking to class", "breaking", "being invited", "getting home", "setting", "dropping", "not seeing", "forgetting", "being called fat", "getting lied", "invited", "to sit here", "to be ignored", "being late", "doing laundry", "being taken", "practicing", "babysitting", "getting hit", "being used", "being used", "being reminded", "when falls", "working all day", "running late", "traveling", "peeing", "being hit", "having practice", "not being invited", "being bored", "stepping", "spending my day", "leaving", "almost getting", "being put", "passing", "being at school", "to study", "going to class", "coughing", "sitting in traffic", "being yelled", "fixing", "burning", "walking to school", "wakin", "seeing people", "being accused", "being up early", "scratches", "texting someone", "being invited places", "receiving", "being grounded", "checking", "getting my ass", "getting back", "getting bitched", "getting treated", "only getting", "reviewing", "sitting alone", "getting screwed", "going there", "getting stared", "calling", "watching scary movies", "getting no sleep", "taking tests", "getting locked", "reading tweets", "teaching", "waking up not", "sounding", "getting made", "sleeping alone", "not feeling", "being surrounded", "editing", "being stood up", "to randomly ask", "getting hacked", "getting texts", "having insomnia", "having homework", "blamed", "showing", "being blamed", "getting bad news", "getting played", "being stood", "scrolling", "being lied too", "being a loner", "going weeks", "being up late", "having class", "failing", "being cussed", "listening to women", "when ignores", "cutting", "bring", "burnt", "getting hate", "coming to school", "sitting here", "waking up early", "being called names", "getting replaced", "having bruises", "closing", "coming back", "getting punched", "getting phone", "spending all day", "being pushed", "spending", "not being able", "waking", "working", "sitting", "walking", "coming home", "living", "being lied", "getting", "coming", "going", "running", "to sit", "being called", "to read", "studying", "paying", "texting", "hearing", "replying", "gettin better", "gettin better", "gettin", "eating", "losing", "listening", "to get up", "finding", "to clean", "being able", "seeing", "to run", "to drive", "to go back", "looking", "taking", "putting", "driving", "to start", "posting", "to pay", "telling me", "ruined", "being woke", "hitting", "laying", "cuddling", "reading", "buying", "cancelled", "sending", "to see pictures", "to find out", "sharing", "finishing", "sweating", "to miss", "hurting"])

In [2]:
x_train,y_train,x_test,y_test = pickle.load(open('march27/dataset_used-includes_train_test_split.pkl','rb'))

In [3]:
categories = ['NOT_SARCASM','SARCASM']

In [4]:
def allindices(string, sub):
    l=[]
    i = string.find(sub)
    while i >= 0:
        l.append(i)
        i = string.find(sub, i + 1)
    return l

In [5]:
# Moment of truth:
results = pickle.load(open('results_summary.pkl','rb'))

In [7]:
for _id,resultset in results.items():
    print(_id)
    for dataset,wordsets in resultset.items():
        predicted = []
        if dataset == 'shereen': 
            print('DatasetSarc')
        else: 
            print('DatasetOther')
        
        orig_paper_pos_expr = wordsets['pos']
        orig_paper_neg_expr = wordsets['neg']
        
        predicted = []
        display(HTML('<b>Positive VPs</b>'))
        for tweet in x_test:
            predicted.append('NOT_SARCASM')
            for posvp in orig_paper_pos_expr:
                if posvp in tweet:
                    predicted[-1] = "SARCASM"
                    break
        print(metrics.classification_report(y_test,predicted,target_names=categories)) 

        predicted = []
        display(HTML('<b>Negative Situations</b>'))
        for tweet in x_test:
            predicted.append('NOT_SARCASM')
            for negsit in orig_paper_neg_expr:
                if negsit in tweet:
                    predicted[-1] = "SARCASM"
                    break
        print(metrics.classification_report(y_test,predicted,target_names=categories)) 

        predicted = []
        display(HTML('<b>Contrast(+VPs, –Situations), Unordered</b>'))
        for tweet in x_test:
            predicted.append('NOT_SARCASM')
            for negsit in orig_paper_neg_expr:
                break_negsit = False
                for posvp in orig_paper_pos_expr:
                    if negsit in tweet and posvp in tweet:
                        predicted[-1] = "SARCASM"
                        break_negsit == True
                        break
                if break_negsit:
                    break
        print(metrics.classification_report(y_test,predicted,target_names=categories)) 

        predicted = []
        display(HTML('<b>Contrast(+VPs, –Situations), Ordered</b>'))
        for tweet in x_test:
            predicted.append('NOT_SARCASM')
            for posvp in orig_paper_pos_expr:
                indices = allindices(tweet,posvp)
                break_posvp = False
                if indices:
                    for negsit in orig_paper_neg_expr:
                        for idx in indices:
                            if negsit in tweet[idx+len(posvp):]:
                                predicted[-1] = "SARCASM"
                                break_posvp == True
                                break
                if break_posvp:
                    break
        print(metrics.classification_report(y_test,predicted,target_names=categories)) 

A
DatasetSarc


             precision    recall  f1-score   support

NOT_SARCASM       0.81      0.93      0.87      1512
    SARCASM       0.47      0.21      0.29       412

avg / total       0.74      0.78      0.75      1924



             precision    recall  f1-score   support

NOT_SARCASM       0.79      0.98      0.87      1512
    SARCASM       0.26      0.02      0.04       412

avg / total       0.67      0.78      0.70      1924



             precision    recall  f1-score   support

NOT_SARCASM       0.79      1.00      0.88      1512
    SARCASM       0.80      0.01      0.02       412

avg / total       0.79      0.79      0.70      1924



             precision    recall  f1-score   support

NOT_SARCASM       0.79      1.00      0.88      1512
    SARCASM       0.80      0.01      0.02       412

avg / total       0.79      0.79      0.70      1924

DatasetOther


...

             precision    recall  f1-score   support

NOT_SARCASM       0.79      1.00      0.88      1512
    SARCASM       0.73      0.03      0.05       412

avg / total       0.78      0.79      0.70      1924

DatasetOther


             precision    recall  f1-score   support

NOT_SARCASM       0.79      0.98      0.88      1512
    SARCASM       0.44      0.07      0.11       412

avg / total       0.72      0.78      0.71      1924



             precision    recall  f1-score   support

NOT_SARCASM       0.80      0.94      0.86      1512
    SARCASM       0.35      0.11      0.17       412

avg / total       0.70      0.76      0.71      1924



             precision    recall  f1-score   support

NOT_SARCASM       0.79      1.00      0.88      1512
    SARCASM       0.25      0.00      0.00       412

avg / total       0.67      0.78      0.69      1924



             precision    recall  f1-score   support

NOT_SARCASM       0.79      1.00      0.88      1512
    SARCASM       0.00      0.00      0.00       412

avg / total       0.62      0.78      0.69      1924



In [6]:
results

{'A': {'scraped': {'discards': {'neg': {'#born #leader',
     '#defeated',
     '#heading',
     '#trump',
     '#willing',
     '#woke',
     'accidentally falling',
     'accommodating them',
     'adulting',
     'affecting',
     'almost passing',
     'assumes he',
     'attracting black',
     'babysitting',
     'backfires',
     'being',
     'being able',
     'being alone',
     'being attacked again',
     'being blanked',
     'being bombarded with',
     'being canadian',
     'being cold',
     'being consistently',
     'being delayed',
     'being disrespected by',
     'being emotional',
     'being emotionally',
     'being good',
     'being hit by',
     'being home',
     'being hung over',
     'being ignored',
     'being ignored whilst',
     'being kept in',
     'being late',
     'being left alone',
     'being left out',
     'being lied to',
     'being low-key',
     'being me',
     'being not',
     'being pale',
     'being peed on',
     'being power',

In [9]:
results['A']['scraped']['discards']['pos']

{'#breeds',
 '#not be',
 '#not enjoy',
 '#not going',
 '#sarcasm keep',
 'absolutely love',
 'allows',
 'allright #not',
 'almost want',
 'always love',
 'am',
 'am always',
 'am beginning',
 'am just',
 'am literally',
 'am not',
 'am really',
 'am totally',
 'am totes',
 'are',
 'are also',
 'are just',
 'are never',
 'are not',
 'are really',
 'are totally',
 "aren't even",
 'asking',
 'assume',
 'atleast am',
 'back then',
 'be',
 'be expected',
 'be trying',
 'been',
 'been there',
 'being',
 'believe',
 'believe is',
 'buffer when',
 'can keep',
 'can start',
 "can't continue",
 "can't wait",
 'cant stop',
 'can’t stand',
 'cares',
 'catch',
 'change',
 'clearly not',
 'collapsed',
 'coming',
 'conflicted',
 'conned',
 'considering',
 'constantly deflecting',
 'could be',
 "couldn't resist",
 'cussing',
 'debating quitting',
 'definitely not',
 'delaying',
 'didn’t need',
 'do',
 "doesn't love",
 "don't have",
 "don't start",
 "don't try",
 'drinking is',
 'else is',
 'else was',

In [10]:
for algo,resultset in results.items():
    print(algo)
    for dataset,wordsets in resultset.items():
        print(dataset)
        print(len(wordsets['pos']))
        print(len(wordsets['neg']))
        print(len(wordsets['discards']['pos']))
        print(len(wordsets['discards']['neg']))

A
scraped
11
24
312
495
shereen
2
20
166
731
B
scraped
18
40
405
1508
shereen
18
46
481
1239
C
scraped
17
18
409
269
shereen
19
35
578
336


In [14]:
for algo,resultset in results.items():
    print(algo)
    for dataset,wordsets in resultset.items():
        print(dataset)
        print(len(wordsets['pos'] & orig_paper_pos_expr))
        print(len(wordsets['neg'] & orig_paper_neg_expr))
        print(len(wordsets['discards']['pos'] & orig_paper_pos_expr))
        print(len(wordsets['discards']['neg'] & orig_paper_neg_expr))

A
scraped
0
8
6
59
shereen
1
10
7
78
B
scraped
0
10
8
65
shereen
1
22
12
94
C
scraped
1
13
10
61
shereen
0
28
14
71


In [17]:
for algo,resultset in results.items():
    print(algo)
    for dataset,wordsets in resultset.items():
        print(dataset)
        #print(len(wordsets['pos'] & orig_paper_pos_expr))
        #print(len(wordsets['neg'] & orig_paper_neg_expr))
        print(wordsets['discards']['pos'] & orig_paper_pos_expr)
        #print(len(wordsets['discards']['neg'] & orig_paper_neg_expr))

A
scraped
{"can't wait", 'excited', 'please keep', 'stops', 'loving', 'love'}
shereen
{'excited', 'really like', 'get', 'enjoy', 'loving', 'missed', 'love'}
B
scraped
{"can't wait", 'break', 'excited', 'please keep', 'get', 'stops', 'loving', 'love'}
shereen
{'loving', "can't wait", 'really like', 'wanted', 'cant wait', 'live', 'get', 'cannot wait', 'looooove', 'loves', 'just stops', 'stoked'}
C
scraped
{'break', 'appreciate', 'excited', 'really like', 'please keep', 'live', 'get', 'stops', 'loving', 'love'}
shereen
{'loving', "can't wait", 'appreciate', 'excited', 'really like', 'just keeps', 'live', 'get', 'missed', 'enjoy', 'looooove', 'loves', 'just stops', 'love'}
